In [1]:
'''
Updated in March 2018; Last updated in Aug 2019 by wz 
@author: A.Karpathy
@editor: WZ
'''
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
You might need to run as well: 
import nltk
nltk.download()
to get the NLTK data and store the data on your your Google Drive.
This demo uses the Drive 
"""

'\nMinimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)\nBSD License\n\nYou might need to run as well: \nimport nltk\nnltk.download()\n\nto get the NLTK data and store the data on your drive \nThis demo uses the drive \n'

Perhaps the easiest way to work with your data is to mount your google drive

In [15]:
from google.colab import drive
drive.mount('/content/drive')
%ls 
%ls ./drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/  sample_data/
'My Drive'/  'Shared drives'/


Getting the data

In [16]:

inputtxt="/content/drive/My Drive/Colab Notebooks/%%Intro2NLP/%Intro2NLPshared/%NLTK_Data_Partial/english-kjv.txt"
data = open(inputtxt, 'r').read() # should be simple plain text file
print(data[0:100])




In the beginning God created the heaven and the earth.
And the earth was without form, and void; and


In [17]:


chars = list(set(data))
print('chars', chars)

chars ['g', '\n', 'Y', 'T', 'Z', 's', 'b', 'p', ' ', 'E', '?', 'J', 'd', 'N', 'B', 'P', 'O', 'q', 'k', 'f', 'y', 'z', 'x', 'G', 't', '!', 'e', 'I', 'o', 'i', 'u', '.', 'm', 'h', ')', 'w', 'r', 'R', "'", 'A', 'c', 'F', 'C', 'l', 'a', 'M', 'U', 'n', 'W', ';', 'H', 'j', 'L', '(', 'S', 'K', ':', 'D', 'v', ',']


In [18]:
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 195515 characters, 60 unique.


In [21]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
print(char_to_ix)

ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(ix_to_char)

{'g': 0, '\n': 1, 'Y': 2, 'T': 3, 'Z': 4, 's': 5, 'b': 6, 'p': 7, ' ': 8, 'E': 9, '?': 10, 'J': 11, 'd': 12, 'N': 13, 'B': 14, 'P': 15, 'O': 16, 'q': 17, 'k': 18, 'f': 19, 'y': 20, 'z': 21, 'x': 22, 'G': 23, 't': 24, '!': 25, 'e': 26, 'I': 27, 'o': 28, 'i': 29, 'u': 30, '.': 31, 'm': 32, 'h': 33, ')': 34, 'w': 35, 'r': 36, 'R': 37, "'": 38, 'A': 39, 'c': 40, 'F': 41, 'C': 42, 'l': 43, 'a': 44, 'M': 45, 'U': 46, 'n': 47, 'W': 48, ';': 49, 'H': 50, 'j': 51, 'L': 52, '(': 53, 'S': 54, 'K': 55, ':': 56, 'D': 57, 'v': 58, ',': 59}
{0: 'g', 1: '\n', 2: 'Y', 3: 'T', 4: 'Z', 5: 's', 6: 'b', 7: 'p', 8: ' ', 9: 'E', 10: '?', 11: 'J', 12: 'd', 13: 'N', 14: 'B', 15: 'P', 16: 'O', 17: 'q', 18: 'k', 19: 'f', 20: 'y', 21: 'z', 22: 'x', 23: 'G', 24: 't', 25: '!', 26: 'e', 27: 'I', 28: 'o', 29: 'i', 30: 'u', 31: '.', 32: 'm', 33: 'h', 34: ')', 35: 'w', 36: 'r', 37: 'R', 38: "'", 39: 'A', 40: 'c', 41: 'F', 42: 'C', 43: 'l', 44: 'a', 45: 'M', 46: 'U', 47: 'n', 48: 'W', 49: ';', 50: 'H', 51: 'j', 52: 'L',

In [0]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [0]:
import numpy as np 

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [0]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

"""   sample a sequence of integers from the model 
h is memory state, seed_ix is seed letter for first time step """

In [0]:
def sample(h, seed_ix, n):
   x = np.zeros((vocab_size, 1))
   x[seed_ix] = 1
   ixes = []
   for t in range(n):
      h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
      y = np.dot(Why, h) + by
      p = np.exp(y) / np.sum(np.exp(y))
      ix = np.random.choice(range(vocab_size), p=p.ravel())
      x = np.zeros((vocab_size, 1))
      x[ix] = 1
      ixes.append(ix) 
   return ixes


In [0]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

### Please don't run it for 12h on colab 


In [0]:
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 10000 == 0:  #was 1000
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print( '----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 10000 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress #was 1000
  
  # perform parameter update with Adagrad
  '''wz
  Returns an iterator of tuples, where the i-th tuple contains the i-th element from each 
  of the argument sequences or iterables. 
  The iterator stops when the shortest input iterable is exhausted. 
  With a single iterable argument, it returns an iterator of 1-tuples.
  '''
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 Zjxo,duYvwZHuM;hOZTx  KUU
ag:!IMx.Pj:FHykAqFybN'UTMeGh.rGOdp?IYz)DDI);ma
RLth(;L s,ZtYGa.cvBkI ;no;EtDKwMh;D,ivtLlqkLWDdCwcgZxhG)eowbGtuFliaOb: qfJ(
fYnBoefRbsDsih);dx,elyUFRgrnqK(lSDFSHeFeEEGWPkB,hS
 
----
iter 0, loss: 102.358615
----
 ld unto shim, ssond theif be tlf, a sum gins reat thes
fart the beroul? for the seed at theiwed f bnemalpon of the dith theu hougdled come, f
 in lighty siwist maked, I his the land
of af eveishtungin 
----
iter 10000, loss: 44.428494
----
 ? and coed and taid kseit Lot tert the bo
came art that
to dixefs
in that thith flaedsaid noto spoked frondss.
And he daccondred snotherd give to the sonk
Ry forth the dloccored; and stood abreg camef 
----
iter 20000, loss: 39.882528
----
 re want in their of the nim, lloodle came that till upon they the eold.
And the
cow, I pree dorth hom untord
your ye bum.
And us not the mad, thes
 it youred upon went go up he ray.
And Bech are told  
----
iter 30000, loss: 38.724414
----
 f the sane the her, and the bough

In [0]:
# prior run sampled

In [0]:
"""
----
 MlpSYgG)PscAJcCF)maYMxgcsAEMeaG'uG!awALhf
nTbnBs'?C?EmdMe''JaLW)fDFH(zHGJdbkGhZNSP:pAWYvaYycGbpo,dx(iMZWUHK
h)zLSqproOICzgvLbGo(.a, ?hK?v
kDwLoWC;PiecyfmGnE:
TLhj),gv
F.k,s;
cfL(IamKwqnNzihWusAFIROgIu 
----
iter 0, loss: 102.358623
----
 ll cworon of foed wa.
c ide wtonAD fanty faJm aol dehibe ,he thee dald y,afas. hil fofe wahan.d; th aindeethet ans mnd eiri heter
ind Nveyy
falnle vathe thallt, url, of arert spy toat and tohy ge meeo 
----
iter 1000, loss: 80.115990
----
 , I unthedering alr thar iled Gof hing thevyrovend and herth ild, yam hes
sorisu.
pet w
se
kat his how le
Ardixd I Hhile, ind
the the
mi goas bime Idind, And mald henling fomay, alaeg doter; wance I f 
----
iter 2000, loss: 65.526007
----
  in noths of chou'd Gean pavenco gid liy the the he elrs thoins min ox.
And the iod go sotoy saaven and ofhim he Noch the ahe on tee in ok thaecem, lhamthe salay.
And sava htht thricarshers
them thae  
----
iter 3000, loss: 57.927522
----
 ---
  wored all don terver,
whot liverwe vift, wuund ban anten.
And mad unto if us thy hes in thou unto her, and case shall not a thren
sputh unto mazind him Abrahah.
And they lill dounst will the king to  
----
iter 50000, loss: 42.217230
----
 
----
 
And the sais
Enou food men behold, spour him Hass in to hit fassy in theirs.
And whous Phar' shau. And Joreh en the cond goed lose, and in on youre, and it tyough.f fore, and to sast
ho, Behold be ao 
----
iter 85000, loss: 40.487233
----
  king is my said, bucass. And he soed satild youl and the bros on Poweph yeom beforest land in shalssed; as at
he notsy be Jacob I alle:
Is
ald.
And rearftour: and they tles wild ald my falleld ha fol 
----
iter 86000, loss: 39.939279
----
----
 may in
Esaughid servaget Isaach the fapleb
the begam, Unar theued and to the pare ase gad anoon bleires not sat at his sons
wast sons thee pord hiveh burost sabob him a fie said, breons
the eald shirk 
----
iter 125000, loss: 39.021318
----
 ed the mon of the earelvang, of will of God, and intt
samal.
And this hast they awale, Beho the comed douth, Pather spay madh they Leew, sseanter onding the lowtord terimese: Fearah of upon.
The
went  
----
iter 126000, loss: 38.998472
----
----
iter 168000, loss: 38.126692
----
 ee his not then, the seas tom shall said, I with silt, Os naml, when Abraht thy seevantsth, and houcher
sthat caso is Abraham's nase teme. butobhes not head came of my
are shailf begat kistrer, unto w 
----
iter 169000, loss: 38.216707
----
pots, reart, and were agired 
----
iter 199000, loss: 37.881385
----
 hen
unto the shall brouse,
fill a lothered hid with me, dake will bearath all ane wiver is forth brock that eves,
and shele upon
the graceed vawat have Rachee inger as Adat in to Jacob be agained so o 
----
iter 200000, loss: 37.349877
----
 d his pin
dreabinge out
of Jadelis, and had up his said, whit I but thele of a feet this in to
will of Hasron.
And ithedd I dweldy mured's wi livedter he said, Whouseart shall Bechel
in the in ano, an 
----
----
iter 218000, loss: 37.791892
----
 hy agoin of the forchemebes Esaurn tham neraned hiigst years
his owered at shall the brepet perdinge him
Esiin remivering, wen, wheirn and the lersin unto the eard
they up to thy giver tole offerd off 
----
iter 219000, loss: 37.282680
----
 's of the king of hie they are
not. Sorn me intt of thy cpae undremo.
And the
youprnas, bort unto
the satelerp, and in that the ray, Learth.
And the LORDarresh of hered is went LORD dreing madefe thou 
----
iter 220000, loss: 37.477838
----
 ilt said there
of wad Abramaen dothy will lad ix the gilt with begai go dimied chanted, and give him upon the meish bach of thounstn his with my may ofly wifl it this sent, as no benceay.
And they, an 

"""

In [0]:
"""
iter 550000, loss: 36.898161
----
 und unto Ezy this wife dow Aritmeath on getter all of in
the pabe wifl caund thou hast thee thems, thee, and we his sweevers a spojame a Laha an the his dost yeak, a place hed him in wath till he twit 
----
iter 560000, loss: 36.471327
----
 en not that were thee, and in bedatr to thin shall ulothwain brether that thes moured their from younsew to Egyptshan dotery sood their
Pherauch after: elf.
And the ear; bocks.
And they senver down wr 
----
iter 570000, loss: 36.587976
----
 toot.
And the nah we hang.
And the edeth and thy tarah of Mahan, and in the yoy were joseld tod the rangass blesshin he self, what shall with heered hlave foundigath in thee; fourd of Ephpraid fivin l 
----
iter 580000, loss: 37.124505
----
 not and divad of him.
And shes which Padany.
And I
pimye
went I
so sunce, and all my wake; and he mad henter was hent one unto theu, and his thous him fatrer of Abron Eeve out shall a with Gone shall  
----
iter 590000, loss: 36.470549
----
 and leen.
And wanct
thlir the cave havivi even of mezangen not feartind.
And she bailneys of in mages thees; be
to their to my foin Sore over plank, betoutsh unto the froh he dieve. Tut the porled opr 
----
iter 600000, loss: 36.895082
"""